# Описание проекта

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Описание данных

## Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя
## Целевой признак<br>
- Price — цена (евро)

# Для выполнения поставленной задачи выполним следующие шаги:
1. Загрузим и подготовим данные.
2. Обучим разные модели с различными гиперпараметрами.
3. Проанализируем скорость работы и качество моделей.

# 1. Откроем файл с данными, изучим его и проведем предобработку

Подключим необходимые библиотеки:

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import time

In [5]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
from sklearn.preprocessing import OrdinalEncoder

In [8]:
from sklearn.metrics import make_scorer

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
from lightgbm import LGBMRegressor

In [11]:
import xgboost as xgb

In [12]:
from catboost import CatBoostRegressor

Загрузим данные в таблицу:

In [13]:
try:
    autos = pd.read_csv('/Users/elenadolgova/Documents/Yandex praktikum/project11/autos.csv')
except:
    autos = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')

Взглянем на первые 5 строк таблицы, чтобы посмотреть какие именно данные в ней представлены.

In [14]:
autos.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Выведем общую информацию о таблице и посмотрим какие типы данных размещены в таблице:

In [15]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Удалим дубликаты сразу (до обработки), так как пока у нас множество колонок, то дубликатов не должно быть много и их удаление не исказит результаты. Далее мы будем работать с сокращенной таблицей и появление там дубликатов вполне возможно даже после удаления дубликатов на этом шаге, но удалять последующие дубликаты мы уже не будем, так как они вызваны просто удалением лишних колонок.

In [16]:
autos.duplicated().sum()

4

In [17]:
autos = autos.drop_duplicates()

Выделим список колонок, которые будут использованы для построения моделей. Нам понадобятся колонки с признаками и целевым признаком. С целевым признаком все просто - это колонка `Price`.

Вряд ли в качестве признаков следует рассматривать следующие колонки, так как они не относятся к характеристикам товара, определяющим его стоимость:

- `DateCrawled` — дата скачивания анкеты из базы
- `RegistrationYear` — год регистрации автомобиля
- `RegistrationMonth` — месяц регистрации автомобиля
- `DateCreated` — дата создания анкеты
- `NumberOfPictures` — количество фотографий автомобиля
- `PostalCode` — почтовый индекс владельца анкеты (пользователя)
- `LastSeen` — дата последней активности пользователя

Сформируем список колонок, с которыми мы будем дальше работать:

In [18]:
columns = ['RegistrationYear', 'Gearbox', 'Power', 'Model', 
           'Kilometer', 'FuelType', 'Brand', 'NotRepaired', 'Price']

Создадим новую таблицу исключительно с интересующими нас колонками и посмотрим на типы данных в них:

In [19]:
auto = autos[columns]

In [20]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   RegistrationYear  354365 non-null  int64 
 1   Gearbox           334532 non-null  object
 2   Power             354365 non-null  int64 
 3   Model             334660 non-null  object
 4   Kilometer         354365 non-null  int64 
 5   FuelType          321470 non-null  object
 6   Brand             354365 non-null  object
 7   NotRepaired       283211 non-null  object
 8   Price             354365 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 27.0+ MB


Как видно, в некоторых колонках есть пропуски, тип данных соответствует описанию колонок. Обработаем колонки.

# 1.1  Обработка колонки `Model`

Пропущенные значения в колонке `Model` мы заполним на основании данных о мощности `Power` и марке авто `Brand`. Для этого сначала создадим сводную таблицу, где расчитаем медианное значение мощности по марке и модели авто.

In [21]:
brand_model_power = auto.pivot_table(index = ['Brand', 'Model'], values = 'Power', aggfunc = 'median')

Затем напишем функцию, которая для пропущенного значения в колонке `Model` на основании сводной таблицы найдет марку и по мощности выдаст модель.

In [22]:
def fill_model(row):
    try:
        if pd.isna(row['Model']):
            if row['Power'] > 0:
                return brand_model_power[brand_model_power['Power'] == row['Power']].index[0][1]
    except:
        return row['Model']
    return row['Model']

Выполним заполнение пропущенных значений.

In [23]:
auto['Model'] = auto.apply(fill_model, axis = 1)

Проверим работу функции.

In [24]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   RegistrationYear  354365 non-null  int64 
 1   Gearbox           334532 non-null  object
 2   Power             354365 non-null  int64 
 3   Model             345153 non-null  object
 4   Kilometer         354365 non-null  int64 
 5   FuelType          321470 non-null  object
 6   Brand             354365 non-null  object
 7   NotRepaired       283211 non-null  object
 8   Price             354365 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 27.0+ MB


Как видно еще остались пропущенные значения в колонке `Model`, но их количество невелико по сравнению с общим количеством строк - их можно удалить. Мы обработаем следующие колонки, а затем проведем удаление уже в конце обработки.

# 1.2 Обработка колонки `NotRepaired`

Данная колонка означает была машина в ремонте или нет, поэтому можно предположить, что если значение там пропущено, значит машина в ремонте не было. Поэтому заменим пропущенные значения на `no`.

In [25]:
auto['NotRepaired'] = auto['NotRepaired'].fillna('no')

# 1.3 Обработка колонки `FuelType`

Посмотрим на уникальные значения в этой колонке и частоту их появления.

In [26]:
auto['FuelType'].value_counts()

petrol      216349
gasoline     98719
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

Как видно, доминируют `petrol` и `gasoline`. Эти два термина означают один и тот же тип двигателя - приведем данные в колонке к одному названию - `petrol`, а поскольку количество пропущенных значений относительно невелико, то не будет сильных искажений, если мы заменим их на самый популярных тип двигателя - `petrol`.

In [27]:
auto['FuelType'] = auto['FuelType'].fillna('petrol')

In [28]:
auto['FuelType'] = auto['FuelType'].replace('gasoline', 'petrol')

# 1.4 Обработка колонки `Gearbox`

Посмотрим на уникальные значения в этой колонке и частоту их появления.

In [29]:
auto['Gearbox'].value_counts()

manual    268249
auto       66283
Name: Gearbox, dtype: int64

Как видно, доминирует ручная коробка - `manual`. Поскольку восстановить по данным из таблицы какая именно коробка у авто невозможно, заменим пропущенные значения на самый популярный вид коробки - `manual`.

In [30]:
auto['Gearbox'] = auto['Gearbox'].fillna('manual')

# 1.5 Удаление пропущенных значений

Мы провели обработку пропущенных значений, поэтому оставшиеся пропущенные значения мы просто удалим.

In [31]:
auto = auto.dropna(subset = columns)

Проверим результат обработки:

In [32]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345153 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   RegistrationYear  345153 non-null  int64 
 1   Gearbox           345153 non-null  object
 2   Power             345153 non-null  int64 
 3   Model             345153 non-null  object
 4   Kilometer         345153 non-null  int64 
 5   FuelType          345153 non-null  object
 6   Brand             345153 non-null  object
 7   NotRepaired       345153 non-null  object
 8   Price             345153 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 26.3+ MB


Как видно, мы удалили около 9 тыс строк, что немного относительно общего количества строк. Данных для построения моделей у нас по-прежнему достаточно.

# 1.6 Проверка выпадающих значений в колонках

Для колонок, в которых количество уникальных значений меньше 100 выведем уникальные значения в этих колонках, чтобы увидеть выпадающие значения. Отсутствие каких-либо колонок в списке внизу укажет на наличие множества уникальных значений в таких колонках - с ними поработаем отдельно.

In [33]:
for col in auto.columns:
    if len(auto[col].unique())<100:
        print(col, '->', auto[col].unique())

Gearbox -> ['manual' 'auto']
Kilometer -> [150000 125000  90000  40000  30000  70000   5000 100000  60000  20000
  80000  50000  10000]
FuelType -> ['petrol' 'lpg' 'other' 'hybrid' 'cng' 'electric']
Brand -> ['volkswagen' 'audi' 'jeep' 'skoda' 'bmw' 'peugeot' 'ford' 'mazda'
 'nissan' 'renault' 'mercedes_benz' 'opel' 'seat' 'citroen' 'honda' 'fiat'
 'mini' 'smart' 'hyundai' 'alfa_romeo' 'subaru' 'volvo' 'mitsubishi' 'kia'
 'suzuki' 'lancia' 'toyota' 'chevrolet' 'dacia' 'daihatsu' 'chrysler'
 'sonstige_autos' 'jaguar' 'daewoo' 'porsche' 'rover' 'saab' 'trabant'
 'land_rover' 'lada']
NotRepaired -> ['no' 'yes']


Как видно, в большинстве колонок вполне соответствующие их описанию значения.

# 1.7 Обработка колонки `Power`

Посмотрим на диапазон значений в колонке.

In [34]:
auto['Power'].describe()

count    345153.000000
mean        111.346904
std         182.492799
min           0.000000
25%          71.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

Как видно, есть нулевые и нереальные 20000 лошадиных сил. Это маловероятно, поэтому исправим такие выбросы. Создадим сводную таблицу где по модели авто посчитаем медианное значение года выпуска `RegistrationYear` (пригодится нам дальше) и мощности `Power`.

In [35]:
models = auto.pivot_table(index = 'Model', values = ['RegistrationYear', 'Power'], aggfunc = 'median')

In [36]:
models = models.to_dict()

Напишем функцию которая обрежет экстремальные значения и поставит вместо них медианное значение мощности по соответствующей модели авто.

In [37]:
def fill_power(row):
    try:
        if (row['Power'] > 400) or (row['Power'] < 75):
            return models['Power'][row['Model']]
        return row['Power']
    except:
        return row['Power']

Произведем замену при помощи функции:

In [38]:
auto['Power'] = auto.apply(fill_power, axis = 1)

Посмотрим какие теперь значения в колонке `Power`.

In [39]:
auto['Power'].describe()

count    345153.000000
mean        119.558983
std          49.792767
min           0.000000
25%          82.000000
50%         109.000000
75%         143.000000
max         400.000000
Name: Power, dtype: float64

Как видно, нулевые значения по-прежнему есть. Посмотрим что это за строки.

In [40]:
auto[auto['Power'] == 0]

,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired,Price
3338,1995,manual,0.0,samara,100000,petrol,lada,no,199
26625,2016,manual,0.0,samara,150000,petrol,lada,no,200
42665,1997,manual,0.0,samara,50000,petrol,lada,yes,300
45401,2007,manual,0.0,samara,125000,petrol,lada,no,0
69358,1995,manual,0.0,samara,125000,petrol,lada,no,99
73120,1991,manual,0.0,samara,70000,petrol,lada,no,2199
75177,1985,manual,0.0,samara,125000,petrol,lada,no,249
100568,1997,manual,0.0,samara,125000,petrol,lada,no,250
150050,2007,manual,0.0,samara,125000,petrol,lada,no,650
201433,1999,manual,0.0,samara,125000,lpg,lada,no,1499


Строк совсем немного и большинство из них относится к одной модели - `samara`. Обычно у этого автомобиля лошадиные силы в районе 80, поэтому произведем замену нуля на 80.

In [41]:
auto['Power'] = auto['Power'].replace(0, 80)

Проверим как осуществилась замена.

In [42]:
auto['Power'].describe()

count    345153.000000
mean        119.562227
std          49.787581
min          26.000000
25%          82.000000
50%         109.000000
75%         143.000000
max         400.000000
Name: Power, dtype: float64

Нулевых значений больше нет, замена осуществлена.

# 1.8 Обработка колонки `RegistrationYear`

Посмотрим на уникальные значения в колонке `RegistrationYear` предварительно отсортировав их по возрастанию для удобства.

In [43]:
years = auto['RegistrationYear'].unique()

In [44]:
years.sort()

In [45]:
years

array([1000, 1001, 1111, 1200, 1234, 1300, 1400, 1500, 1600, 1602, 1800,
       1910, 1919, 1923, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934,
       1935, 1936, 1937, 1938, 1941, 1942, 1943, 1945, 1947, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2066, 2200, 2290, 2500, 2900, 3000, 3500, 3700,
       4000, 4500, 5000, 5555, 5900, 5911, 6000, 6500, 7000, 7100, 7500,
       7800, 8000, 8200, 8500, 9000, 9999])

Как видно, есть года, которые явно выпадают из списка ожидаемых. Посмотрим на популярный диапазон - не старше 1960 года, но не моложе 2019, так как данных за 2020 год нет в таблице.

In [46]:
len(auto[(auto['RegistrationYear'] > 2019) | (auto['RegistrationYear'] < 1960)])

328

Количество строк вне популярного диапазона невелико - заполним их медианным значением года выпуска по модели авто. Для это создадим сводную таблицу и посчитаем в ней медианное значение года выпуска `RegistrationYear` для каждой модели авто. Такой подход не внесет сильных искажений. Сама сводная таблица уже построена нами в предыдущем пункте и имеет название `models`.

Напишем функцию, которая для значений вне популярного диапазона заменит года на медианное значение года выпуска по модели авто.

In [47]:
def fill_year(row):
    if (row['RegistrationYear'] > 2019) or (row['RegistrationYear'] < 1960):
        return models['RegistrationYear'][row['Model']]
    return row['RegistrationYear']

Произведем замену значений при помощи функции.

In [48]:
auto['RegistrationYear'] = auto.apply(fill_year, axis = 1)

Посмотрим как выглядит таблица после исправлений:

In [49]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345153 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   RegistrationYear  345153 non-null  float64
 1   Gearbox           345153 non-null  object 
 2   Power             345153 non-null  float64
 3   Model             345153 non-null  object 
 4   Kilometer         345153 non-null  int64  
 5   FuelType          345153 non-null  object 
 6   Brand             345153 non-null  object 
 7   NotRepaired       345153 non-null  object 
 8   Price             345153 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 26.3+ MB


## Вывод:

Пропущенных значений не содержится, а типы данных соответствуют их описанию.

# 2. Обучение моделей

Далее мы будем использовать модели для бустинга - LGBMRegressor, XGBoost и CatBoost. Для каждой модели у нас будет один набор гиперпараметров с небольшим диапазоном значений - это позволит модели подобрать лучшие гиперпараметры, а нам сравнить время на обучение и итоговую точность моделей.

# 2.1 Построение модели при помощи LGBMRegressor

Для работы с LGBMRegressor требуется указать какие именно колонки в таблице являются категориальными, то есть описательными, а не численными. Кроме того, формат данных в такой колонке должен иметь тип `category`. Соберем колонки с категориальными данными в отдельный список и для каждого элемента этого списка произведем смену типа данных на `category`.

In [50]:
cat_features = ['Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

In [51]:
for item in cat_features:
    auto[item] = auto[item].astype('category')

Выделим признаки в отдельный список и разобъем данные таблицы `auto` на обучающую и тестовую выборки. Тестовую мы будем использовать, чтобы посчитать оценить метрику. В качестве метрики мы выберем `RMSE`. Чем меньше будет значение `RMSE`, тем лучше модель предсказывает стоимость авто.

In [52]:
features = ['RegistrationYear', 'Gearbox', 'Power', 'Model', 'Kilometer', 'FuelType', 'Brand', 'NotRepaired']

In [53]:
features_train, features_test, target_train, target_test = (train_test_split(
    auto[features], auto['Price'], test_size = 0.25, random_state = 123))

Создадим объект модели.

In [54]:
estimator = LGBMRegressor(random_state=123)

Мы уже знакомы с некоторыми параметрами, которые могут сильно повлиять на результат обучения. Это 
- максимальная глубина дерева - `max_depth`
- скорость обучения - `learning_rate`
- количество деревьев в ансамбле - `n_estimators`

Для них установим диапазон и шаг в следующем словаре:

In [55]:
param_grid = {'max_depth': range(10, 20, 5), 
          'learning_rate': np.arange(0, 0.5, 0.25), 
          'n_estimators': range(250, 400, 50)}

Создадим метрику для использования в `GridSearchCV`.

In [56]:
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [57]:
gbm = GridSearchCV(estimator, param_grid, cv=3, scoring = mse_scorer)

In [58]:
g_start = time.time()
gbm.fit(features_train, target_train)
g_end = time.time()
gbm_time = g_end - g_start

Посмотрим при каких параметрах достигнуто лучшее значение метрики `RMSE`

In [59]:
gbm.best_params_

{'learning_rate': 0.25, 'max_depth': 15, 'n_estimators': 350}

Посчитаем значение `RMSE` на тестовой выборке:

In [60]:
gbm_rmse = int(sqrt(mean_squared_error(target_test, gbm.predict(features_test))))

In [61]:
gbm_rmse

1794

Значения времени исполнения кода на подбор параметров и значение `RMSE` на тестовой выборке мы сохранили в переменных, чтобы затем свести их в таблицу и сравнить модели между собой.

# 2.2 Построение модели при помощи XGBoost

Для XGBoost требуется кодировать категориальные колонки. Сделаем это при помощи порядкового кодирования - так будет меньше колонок.

In [62]:
encoder = OrdinalEncoder()

In [63]:
cat_features_ordinal = encoder.fit_transform(features_train[cat_features])

Обернем новые трансформированные колонки в `DataFrame`

In [64]:
cat_features_ordinal = pd.DataFrame(data = cat_features_ordinal, columns = cat_features)

Удалим старые колонки с категориальными данными

In [65]:
one = features_train.drop(cat_features, axis = 1).reset_index(drop = True)

Создадим отдельную таблицу по аналогии с `features_train`, но уже с кодированными колонками - для этого объединим исходную таблицу без кодированных колонок и кодированные колонки.

In [66]:
features_train_ordinal = one.merge(cat_features_ordinal, left_index=True, right_index=True)

Будем использовать `GridSearchCV` как и в предыдущем пункте

In [67]:
regressor = xgb.XGBRegressor(random_state = 123)

In [68]:
xgb = GridSearchCV(regressor, param_grid, cv=3, scoring = mse_scorer)

Для расчета `RMSE` на тестовой выборке нам потребуется также закодировать категориальные колонки.

In [69]:
features_test_ordinal = encoder.fit_transform(features_test[cat_features])

In [70]:
features_test_ordinal = pd.DataFrame(data = features_test_ordinal, columns = cat_features)

In [71]:
two = features_test.drop(cat_features, axis = 1).reset_index(drop = True)

In [72]:
features_test_ordinal = two.merge(features_test_ordinal, left_index=True, right_index=True)

In [73]:
x_start = time.time()
xgb.fit(features_train_ordinal, target_train)
x_end = time.time()
xgb_time = x_end - x_start

Посмотрим при каких параметрах достигнуто лучшее значение метрики `RMSE`

In [74]:
xgb.best_params_

{'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 250}

Посчитаем значение `RMSE` на тестовой выборке:

In [75]:
xgb_rmse = int(sqrt(mean_squared_error(target_test, xgb.predict(features_test_ordinal))))

Посмотрим на время работы бустинга в минутах

In [76]:
int(xgb_time/60)

25

# 2.3 Построение модели при помощи CatBoost

Для CatBoost не требуется производить никакого кодирования признаков, поэтому сразу приступим к построению модели.

In [77]:
catregressor = CatBoostRegressor(cat_features=cat_features, random_state = 123)

In [78]:
catbst = GridSearchCV(catregressor, param_grid, cv=3, scoring = mse_scorer)

In [79]:
c_start = time.time()
catbst.fit(features_train, target_train, verbose = 0)
c_end = time.time()
cb_time = c_end - c_start

Посмотрим при каких параметрах достигнуто лучшее значение метрики `RMSE`

In [80]:
catbst.best_params_

{'learning_rate': 0.25, 'max_depth': 15, 'n_estimators': 250}

Посчитаем значение `RMSE` на тестовой выборке:

In [81]:
cb_rmse = int(sqrt(mean_squared_error(target_test, catbst.predict(features_test))))

In [82]:
cb_rmse

1802

Посмотрим на время работы бустинга в минутах

In [83]:
int(cb_time/60)

36

# 2.4 Построение константной модели

Для проверки адекватности всех моделей построим константную модель, которая просто предсказывает среднее значение стоимости авто -  `Price`.

Построим вектор предсказаний по среднему из обучающей выборки и длиною, равной тестовой выборке.

In [84]:
mean_pred = np.tile(target_train.mean(), len(target_test))

Посчитаем `RMSE` для тестовой выборки для константной модели.

In [85]:
random_rmse = int(sqrt(mean_squared_error(target_test, mean_pred)))

In [86]:
random_rmse

4509

Анализ результатов `RMSE` по всем моделям и время расчета проведем на следующем шаге.

# 3. Анализ моделей

Для удобства соберем результаты моделей в таблицу:

In [88]:
score = pd.DataFrame({ 'Модель': ['LightGBM', 'XGBoost', 'CatBoost', 'Среднее'],
                      'RMSE': [gbm_rmse, xgb_rmse, cb_rmse, random_rmse],
                      'Время в мин': [int(gbm_time/60), int(xgb_time/60), int(cb_time/60), 0]
    
})

Посмотрим на результаты:

In [89]:
score

,Модель,RMSE,Время в мин
0,LightGBM,1794,0
1,XGBoost,1860,25
2,CatBoost,1802,36
3,Среднее,4509,0


Как видно, наиболее точные результаты у `LightGBM` и у него же самое короткое время расчета. По времени исполнения с ним сравнится только константная модель, но у такой модели ошибка будет больше в 2 раза. Из дополнительного времени на бустинг надо учесть подготовку данных для `LightGBM`, но она не слишком затруднительна - достаточно указать тип колонки `category`.

Примерно одинаковую точность показывают `XGBoost` и `CatBoost`, но время расчетов `CatBoost` в полтора раза больше, чем у `XGBoost`. Однако для `XGBoost` нужно отдельно кодировать категориальные колонки и делать это два раза - для обучающей и тестовой выборки отдельно, чтобы избежать утечки целевого признака. `CatBoost` не требует обработки, зато требует терпения ждать пока он все посчитает.

Учитывая эти данные можно составить рейтинг моделей:<br>Первое место - `LightGBM` - за свою точность, скорость и неприхотливость в обработке категориальных колонок<br>Второе место - `XGBoost` - за свою скорость<br>Третье место - `CatBoost` - за свою нерасторопность<br>Четвертое место - константная модель - за свою неточность

## Вывод:

Наиболее оптимальным алгоритмом бустинга по показателям точности и скорости следует выбрать `LightGBM`.

## В ходе работы мы проделали следующие шаги:

1. Загрузили и подготовили данные.
2. Обучили разные модели с различными гиперпараметрами.
3. Проанализировали скорость работы и качество моделей.

## На основе проделанной работы сделали выводы:

1. Наиболее точные результаты у `LightGBM` и у него же самое короткое время расчета. По времени исполнения с ним сравнится только константная модель, но у такой модели ошибка будет больше в 2 раза. Из дополнительного времени на бустинг надо учесть подготовку данных для `LightGBM`, но она не слишком затруднительна - достаточно указать тип колонки `category`.

2. Примерно одинаковую точность показывают `XGBoost` и `CatBoost`, но время расчетов `CatBoost` в полтора раза больше, чем у `XGBoost`. Однако для `XGBoost` нужно отдельно кодировать категориальные колонки и делать это два раза - для обучающей и тестовой выборки отдельно, чтобы избежать утечки целевого признака. `CatBoost` не требует обработки, зато требует терпения ждать пока он все посчитает.

3. Наиболее оптимальным алгоритмом бустинга для определения стоимости авто по показателям точности и скорости следует выбрать `LightGBM` за свою точность, скорость и неприхотливость в обработке категориальных колонок.